# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
import joblib
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to /Users/zacks/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/zacks/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/zacks/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql('select * from disaster_response', con=engine)
X = df.message
Y = df.loc[:, 'related':]
target_names = Y.columns
y = Y.to_numpy()

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(message, stem='lemm'):
    """Text processing.
    
    Args:
        stem(str): stem or lemm.
        
    Returns:
        list: Cleaned tokens.
    """
    # 1. Cleaning
    
    # 2. Normalization
    text = re.sub(r"[^a-zA-Z0-9]", " ", message.lower())
    
    # 3. Tokenization
    tokens = word_tokenize(text)
    
    # 4. Stop Word Removal
    stop_words = stopwords.words("english")
    tokens = list(filter(lambda w: w not in stop_words, tokens))
    
    # 5. Part of Speech Tagging / Named Entity Recognition
    
    # 6. Stemming or Lemmatization
    # Because the targets are not roots, we should use Lemmatization
    
    clean_tokens = []
    if stem == 'stem':
        stemmer = PorterStemmer()
        for tok in tokens:
            clean_tok = stemmer.stem(tok).strip()
            clean_tokens.append(clean_tok)
    else:
        lemmatizer = WordNetLemmatizer()
        for tok in tokens:
            clean_tok = lemmatizer.lemmatize(tok).strip()
            clean_tokens.append(clean_tok)

    return clean_tokens

In [4]:
# Test
# eat - starving, water - thirsty should be the same meanings.
# Therefore, the ML models should build the correct word embeddings.
print('Message:', X[10])
print('Targes:', Y.columns.to_list())

Message: There's nothing to eat and water, we starving and thirsty.
Targes: ['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [5]:
print('Stemming', tokenize(X[10], stem='stem'))
print('Lemmatization', tokenize(X[10], stem='lemm'))

Stemming ['noth', 'eat', 'water', 'starv', 'thirsti']
Lemmatization ['nothing', 'eat', 'water', 'starving', 'thirsty']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
# Neural Network needs much more time for modeling
# mlp = MLPClassifier(random_state=42, max_iter=200, verbose=True, early_stopping=True)
vect = TfidfVectorizer(tokenizer=tokenize, use_idf=True, smooth_idf=True)
svd = TruncatedSVD(random_state=42)
forest = RandomForestClassifier(random_state=42, n_jobs=16)
multi_clf = MultiOutputClassifier(forest)

pipeline = Pipeline([
    ('vect', vect),
    ('multi_clf', multi_clf)
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [9]:
for i in range(y_pred.shape[1]):
    print(target_names[i])
    print(classification_report(y_test[:, i], y_pred[:, i]))
    print('-'*60)

related
              precision    recall  f1-score   support

           0       0.71      0.42      0.53      1266
           1       0.83      0.94      0.88      3938
           2       0.39      0.45      0.42        40

    accuracy                           0.81      5244
   macro avg       0.64      0.60      0.61      5244
weighted avg       0.80      0.81      0.79      5244

------------------------------------------------------------
request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.83      0.48      0.61       895

    accuracy                           0.89      5244
   macro avg       0.87      0.73      0.77      5244
weighted avg       0.89      0.89      0.88      5244

------------------------------------------------------------
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classificat

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5177
           1       0.00      0.00      0.00        67

    accuracy                           0.99      5244
   macro avg       0.49      0.50      0.50      5244
weighted avg       0.97      0.99      0.98      5244

------------------------------------------------------------
other_infrastructure
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5021
           1       0.00      0.00      0.00       223

    accuracy                           0.96      5244
   macro avg       0.48      0.50      0.49      5244
weighted avg       0.92      0.96      0.94      5244

------------------------------------------------------------
weather_related
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      3806
           1       0.84      0.69      0.76      1438

    accuracy           

In [10]:
scores = []
for i in range(y_pred.shape[1]):
    scores.append(recall_score(y_test[:, i], y_pred[:, i], average='weighted'))

print('Average Recall:', np.mean(scores))

Average Recall: 0.9482795152131536


### 6. Improve your model
Use grid search to find better parameters. 

> [model_tuning_KNN.ipynb](./model_tuning_KNN.ipynb): 0.9385700059327063<br>
> [model_tuning_randomforest.ipynb](./model_tuning_randomforest.ipynb): 0.9410702178150692<br>
> [model_tuning_MLP.ipynb](./model_tuning_MLP.ipynb): 0.9500010594118147

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
def ml_pipeline(params):
    # text processing and model pipeline
    vect = TfidfVectorizer(tokenizer=tokenize, use_idf=True, smooth_idf=True)
    svd = TruncatedSVD(random_state=42)
    mlp = MLPClassifier(random_state=42, early_stopping=True, learning_rate='adaptive')
    
    multi_clf = MultiOutputClassifier(mlp)

    pipeline = Pipeline([
        ('vect', vect),
        # ('svd', svd),
        ('multi_clf', multi_clf)
    ])
    
    pipeline.set_params(**params)
    
    return pipeline

In [12]:
best_params = {'vect__max_features': 400,
    'multi_clf__estimator__activation': 'relu',
    'multi_clf__estimator__solver': 'adam',
    'multi_clf__estimator__hidden_layer_sizes': 170,
    'multi_clf__estimator__max_iter': 220}

In [13]:
pipeline = ml_pipeline(best_params)

In [14]:
pipeline.set_params(**best_params)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [15]:
scores = []
for i in range(y_pred.shape[1]):
    scores.append(recall_score(y_test[:, i], y_pred[:, i], average='weighted'))

print('Average Recall:', np.mean(scores))

Average Recall: 0.9500010594118147


There is 1.14% performance improvement (recall score).

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

> [model_tuning_KNN.ipynb](./model_tuning_KNN.ipynb): 0.9385700059327063<br>
> [model_tuning_randomforest.ipynb](./model_tuning_randomforest.ipynb): 0.9410702178150692<br>
> [model_tuning_MLP.ipynb](./model_tuning_MLP.ipynb): 0.9500010594118147

### 9. Export your model as a pickle file

In [16]:
joblib.dump(pipeline, 'model.pkl')

['model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.